In [2]:
#To load automatically updates from libraries without re-running import cells
%load_ext autoreload
%autoreload 2

In [ ]:
import cv2
import numpy as np
from multi_shape_detector import MultiShapeDetector
from segmentation_utils import draw_detection_bound_boxes
from segmentation_utils import get_clusters_from_grayscale_image

# Load the test image
image = cv2.imread(f"man_with_dog_and_square.png", cv2.IMREAD_GRAYSCALE)

# Extraxct the object pixels from image
grouped_clusters = get_clusters_from_grayscale_image(image)


# Define human shape image for detector init
human_shape_image = cv2.imread(f"human_shape.png", cv2.IMREAD_GRAYSCALE)
human_detect_params = {
    "label": "person",
    "detect_thresh": 0.75,
    "sigm_params": (0.7, 10),
}

# Define dog shape image for detector init
dog_shape_image = cv2.imread(f"dog_shape.png", cv2.IMREAD_GRAYSCALE)
dog_detect_params = {
    "label": "dog",
    "detect_thresh": 0.75,
    "sigm_params": (0.7, 10),
}

# Init multi-shape detector
multi_shape_detector = MultiShapeDetector(
    [human_shape_image, dog_shape_image], [human_detect_params, dog_detect_params]
)

# Testing pixel clusters 
detections_data = []
for cluster in grouped_clusters:
    cluster_box_data = multi_shape_detector.compare(cluster)
    if len(cluster_box_data) > 0:
        detections_data.append(cluster_box_data)

print(detections_data)

# Draw detection result boxes on image
detect_boxes_image = draw_detection_bound_boxes(image, detections_data)

# Display the result
cv2.imshow("Image", detect_boxes_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[{'color': [0, 255, 0], 'coords_data': (49, 25, 100, 348), 'label': 'person', 'confidence': 0.9129454566308621}, {'color': [0, 255, 0], 'coords_data': (178, 181, 177, 181), 'label': 'dog', 'confidence': 0.9691790398204049}]
